# GRU을 이용한 텍스트 생성
- 데이터 다운로드 캐글https://www.kaggle.com/aashita/nyt-comments?select=ArticlesApril2018.csv
- ArticlesApril2018.csv

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [2]:
seed= 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0]


Saving ArticlesApril2018.csv to ArticlesApril2018.csv


In [4]:
import pandas as pd
df = pd.read_csv(filename, encoding='latin1')
df.head(2)

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleadersâ Settlement Offer...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"âI understand that they could meet with us, ...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...


### 데이터 전처리

In [5]:
print('열의개수: ', len(df.columns))
print(df.columns)

열의개수:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [6]:
df['headline'].isnull().values.any()

False

In [7]:
headline = [text for text in df.headline.values]
headline[:5]

['Former N.F.L. Cheerleadersâ\x80\x99 Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [8]:
len(headline)

1324

In [9]:
# 노이즈데이터(Unknown)제거
headline = [text for text in headline if text !='Unknown']
len(headline)

1214

In [10]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [11]:
# 구두점 제거와 소문자화를 위한함수
def preprocessing(s):
  s=s.encode("utf8").decode("ascii", 'ignore')
  return ''.join(c for c in s if c not in punctuation).lower()

In [12]:
headlines = [preprocessing(x) for x in headline]
headlines[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

In [18]:
# 단어 집합(vocabulary)을 만들
t = Tokenizer()
t.fit_on_texts(headlines)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 3494


In [19]:
sequences = []
for line in headlines:
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
      sequence =encoded[:i+1]
      sequences.append(sequence)

sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [20]:
index_to_word = {}
for key, value in t.word_index.items():
  index_to_word[value] = key

print('빈도수 상위 1번 단어 :' , index_to_word[1])
print('빈도수 상위 582번 단어 :', index_to_word[582])

빈도수 상위 1번 단어 : the
빈도수 상위 582번 단어 : offer


In [21]:
max_len = max(len(s) for s in sequences)
print('샘플의 최대 길이 :', max_len)

샘플의 최대 길이 : 24


In [23]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


In [26]:
X = sequences[:,:-1]
y = sequences[:,-1]

In [27]:
Y = to_categorical(y, num_classes=vocab_size)

In [30]:
X.shape, Y.shape

((7803, 23), (7803, 3494))

### 모델 정의/설정/학습

In [31]:
from keras.models import Sequential
from keras.layers import GRU, Embedding, Dense

In [33]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1, 
                    name="Embedding_Layer"))
model.add(GRU(128, name="GRU_Layer"))
model.add(Dense(vocab_size, activation='softmax', name="Output_Layer"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, 23, 10)            34940     
_________________________________________________________________
GRU_Layer (GRU)              (None, 128)               53760     
_________________________________________________________________
Output_Layer (Dense)         (None, 3494)              450726    
Total params: 539,426
Trainable params: 539,426
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [36]:
hitstory = model.fit(X, Y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 9s - loss: 7.7056 - accuracy: 0.0270
Epoch 2/200
244/244 - 1s - loss: 7.1280 - accuracy: 0.0319
Epoch 3/200
244/244 - 1s - loss: 6.9165 - accuracy: 0.0447
Epoch 4/200
244/244 - 1s - loss: 6.6963 - accuracy: 0.0497
Epoch 5/200
244/244 - 1s - loss: 6.4518 - accuracy: 0.0534
Epoch 6/200
244/244 - 1s - loss: 6.1823 - accuracy: 0.0597
Epoch 7/200
244/244 - 1s - loss: 5.9007 - accuracy: 0.0648
Epoch 8/200
244/244 - 1s - loss: 5.6149 - accuracy: 0.0696
Epoch 9/200
244/244 - 1s - loss: 5.3410 - accuracy: 0.0768
Epoch 10/200
244/244 - 1s - loss: 5.0809 - accuracy: 0.0898
Epoch 11/200
244/244 - 1s - loss: 4.8400 - accuracy: 0.1060
Epoch 12/200
244/244 - 1s - loss: 4.6172 - accuracy: 0.1262
Epoch 13/200
244/244 - 1s - loss: 4.4119 - accuracy: 0.1581
Epoch 14/200
244/244 - 1s - loss: 4.2227 - accuracy: 0.1838
Epoch 15/200
244/244 - 1s - loss: 4.0408 - accuracy: 0.2118
Epoch 16/200
244/244 - 1s - loss: 3.8778 - accuracy: 0.2364
Epoch 17/200
244/244 - 1s - loss: 3.7171 - accura

In [37]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = np.argmax(model.predict(encoded), axis=-1)
          # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장

    sentence = init_word + sentence
    return sentence

In [38]:
print(sentence_generation(model, t, 'i', 10))
print(sentence_generation(model, t, 'how', 10))
print(sentence_generation(model, t, 'former', 10))

i want to be rich and im not sorry us in
how to prevent a racist hoodie comments of new york strike
former nfl cheerleaders settlement offer 1 and a meeting with goodell


### simpleRNN 사용한 경우

In [39]:
from tensorflow.keras.layers import SimpleRNN
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1, 
                    name="Embedding_Layer"))
model.add(SimpleRNN(128, name="SimpleRNN_Layer"))
model.add(Dense(vocab_size, activation='softmax', name="Output_Layer"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, 23, 10)            34940     
_________________________________________________________________
SimpleRNN_Layer (SimpleRNN)  (None, 128)               17792     
_________________________________________________________________
Output_Layer (Dense)         (None, 3494)              450726    
Total params: 503,458
Trainable params: 503,458
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [41]:
history = model.fit(X, Y, epochs=200, verbose=2)

Epoch 1/200
244/244 - 4s - loss: 7.6585 - accuracy: 0.0261
Epoch 2/200
244/244 - 3s - loss: 7.1557 - accuracy: 0.0334
Epoch 3/200
244/244 - 3s - loss: 6.9595 - accuracy: 0.0428
Epoch 4/200
244/244 - 4s - loss: 6.7292 - accuracy: 0.0496
Epoch 5/200
244/244 - 3s - loss: 6.4419 - accuracy: 0.0565
Epoch 6/200
244/244 - 3s - loss: 6.1363 - accuracy: 0.0614
Epoch 7/200
244/244 - 3s - loss: 5.8470 - accuracy: 0.0675
Epoch 8/200
244/244 - 4s - loss: 5.5720 - accuracy: 0.0724
Epoch 9/200
244/244 - 3s - loss: 5.3172 - accuracy: 0.0843
Epoch 10/200
244/244 - 4s - loss: 5.0639 - accuracy: 0.0996
Epoch 11/200
244/244 - 4s - loss: 4.8232 - accuracy: 0.1212
Epoch 12/200
244/244 - 4s - loss: 4.5889 - accuracy: 0.1524
Epoch 13/200
244/244 - 4s - loss: 4.3600 - accuracy: 0.1785
Epoch 14/200
244/244 - 3s - loss: 4.1393 - accuracy: 0.2117
Epoch 15/200
244/244 - 3s - loss: 3.9232 - accuracy: 0.2486
Epoch 16/200
244/244 - 3s - loss: 3.7219 - accuracy: 0.2854
Epoch 17/200
244/244 - 4s - loss: 3.5273 - accura

In [42]:
print(sentence_generation(model, t, 'i', 10))
print(sentence_generation(model, t, 'how', 10))
print(sentence_generation(model, t, 'former', 10))

i want to be rich and im not sorry officials say
how do you feel about being told to smile obstacles by
former nfl cheerleaders settlement offer 1 and a meeting with goodell
